In [ ]:
#### Code to find coordinates showing video for Open field test - shows the certainty of classification and creates a track image

import cv2
import sys 
import math
import csv
import pandas as pd
import os
from shapely.geometry import Polygon, Point
import numpy as np
import shutil
import time

import object_detection_55
from object_detection_55 import ObjectDetection

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

video_list_path = list(clear_filepath("videos/to_analyse/OF_mir"))
print(video_list_path)

od= ObjectDetection()

def dist(pt1, pt2):
    return(math.hypot(pt2[0]-pt1[0],pt2[1]-pt1[1]))

past1 = {}
past2 = {}
past3 = {}
past4 = {}
past5 = {}
past6 = {}
past7 = {}
past8 = {}
past9 = {}
past10 = {}
past_list = [past1, past2, past3, past4, past5, past6, past7, past8, past9, past10]

camera_coords = pd.read_csv("Boundary_coords.csv")

lid_df = pd.read_csv("OF_mirror/Lid_coords.csv")
lid_vids = list(pd.unique(lid_df.Name))

print(time.localtime()[3], ":", time.localtime()[4], ":", time.localtime()[5])

for video in video_list_path :
    
    file_name = os.path.basename(video)
    img_name = file_name.removesuffix(".mp4") + ".jpg"
    track_name = file_name.removesuffix(".mp4") + "_" + "track" + ".jpg"
    dest_img = ("OF_mirror/track_images/" + track_name)
    dest_backup = ("OF_mirror/track_images_clean/" + track_name)
    
    l = [idx for idx, item in enumerate(file_name.lower()) if "_" in item]
    start_pos = l[0] + 1
    end_pos = l[1]
    pos_name = file_name[start_pos:end_pos]
    print(pos_name)
    
    if img_name in lid_vids:
        lid_coords = lid_df[lid_df.Name == img_name]
    
        red_list=[]
        temp = list(lid_coords.Coordinates[lid_coords.Point == "red"])
        for n in temp:
            n=n.replace("(","")
            n=n.replace(")","")
            n=n.split(",")
            if n[0] == "'NA'":
                red_list.append("NA")
            else:
                n[0] = int(n[0])
                n[1] = int(n[1])
                red_list.append(n)

        blue_list=[]
        temp = list(lid_coords.Coordinates[lid_coords.Point == "blue"])
        for n in temp:
            n=n.replace("(","")
            n=n.replace(")","")
            n=n.split(",")
            if n[0] == "'NA'":
                blue_list.append("NA")
            else:
                n[0] = int(n[0])
                n[1] = int(n[1])
                blue_list.append(n)    
    else:
        red_list = ["NA","NA","NA","NA"]
        blue_list = ["NA","NA","NA","NA"]
    
    df = pd.DataFrame(columns=["Video", "Width", "Frames", "Frame", "Box", "Fish", "x_coord", "y_coord", "Mirror", "Lid_pt",
                               "Lid_box", "Certainty"])
    
    df_check = pd.DataFrame(columns=["Video", "Frame", "Box"])
    
    print(file_name)
    cap = cv2.VideoCapture(video)
    
    box_coords=[]
    temp = list(camera_coords.Coordinates[camera_coords.Point == "box"])
    if len(temp) == 0:
        print(file_name, "missing boundaries")
        cap.release()
        continue
        
    for n in temp:
        n=n.replace("(","")
        n=n.replace(")","")
        n=n.split(",")
        n[0] = int(n[0])
        n[1] = int(n[1])
        box_coords.append(n)
    
    mirror=[]
    temp = list(camera_coords.Coordinates[camera_coords.Point == "mirror"])
    if len(temp) == 0:
        print(file_name, "missing boundaries")
        cap.release()
        continue
        
    for n in temp:
        n=n.replace("(","")
        n=n.replace(")","")
        n=n.split(",")
        n[0] = int(n[0])
        n[1] = int(n[1])
        mirror.append(n)
    
    test=[]
    for n in box_coords:
        n=(float(n[0]),float(n[1]))
        test.append(n)
    
    bound_tl=test[0:4]
    bound_tr=test[4:8]
    bound_bl=test[8:12]
    bound_br=test[12:16]
    
    #Polygons for seeing if point within, array for showing on video
    polygon_tl=Polygon(bound_tl)
    polygon_tr=Polygon(bound_tr)
    polygon_bl=Polygon(bound_bl)
    polygon_br=Polygon(bound_br)
    
    polygon_dict = {1: (polygon_tl), 2: (polygon_tr), 3: (polygon_bl), 4: (polygon_br)}
    
    tl_array = np.array([bound_tl], np.int32)
    tr_array = np.array([bound_tr], np.int32)
    bl_array = np.array([bound_bl], np.int32)
    br_array = np.array([bound_br], np.int32)
    
    array_list = (tl_array, tr_array, bl_array, br_array)
    
    test=[]
    for n in mirror:
        n=(float(n[0]),float(n[1]))
        test.append(n)
    
    mir1_coord=test[0:4]
    mir2_coord=test[4:8]
    mir3_coord=test[8:12]
    mir4_coord=test[12:16]
    
    mir1_array=np.array([mir1_coord], np.int32)
    mir2_array=np.array([mir2_coord], np.int32)
    mir3_array=np.array([mir3_coord], np.int32)
    mir4_array=np.array([mir4_coord], np.int32)
    
    mir1_poly=Polygon(mir1_coord)
    mir2_poly=Polygon(mir2_coord)
    mir3_poly=Polygon(mir3_coord)
    mir4_poly=Polygon(mir4_coord)
    
    mir_dict = {1: mir1_array, 2: mir2_array, 3: mir3_array, 4: mir4_array}
    
    
    coord_dict={1:(polygon_tl, mir1_poly, red_list[0], blue_list[0]), 2:(polygon_tr, mir2_poly, red_list[1], blue_list[1]), 
                3:(polygon_bl, mir3_poly, red_list[2], blue_list[2]), 4:(polygon_br, mir4_poly, red_list[3], blue_list[3])}
    
    used_IDs = []
    
    count = 0
    track_id=1
    prev_detections=[]
    tracking_objects={}
    object_trail={}
    
    while True:
        
        ret, frame= cap.read()
        count = count + 1
        if not ret:
            break
        
        current_detections=[]
        tracking_objects_boxes={}
        
        (class_ids, scores, boxes) =od.detect(frame)
        box_nr = 0
        
        width, height = frame.shape[1], frame.shape[0]
        if width == 1920:
            buffer = 22
        elif width == 2560:
            buffer = 38
        
        for box in boxes:
            (x,y,w,h)=box
            
            cx= x+w/2
            cy= y+h/2
            cv2.circle(frame,(int(cx),int(cy)),5,(0,0,120),-1)  
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,120),2)
            
            #cy_new = (cap.get(4)-(y+h/2))
            
            current_detections.append([round(scores[box_nr],2),(cx,cy),(x,y,w,h)])
            box_nr+=1
        
        for container in array_list:
            cv2.polylines(frame, [container], True, (0,120,0))
        
        for key, mirror in mir_dict.items():
            cv2.polylines(frame, [mirror], True, (0,0,120))
        
        for key, item in coord_dict.items():
            (box, mir, red, blue) = item
            if red != "NA":
                x,y = red
                cv2.circle(frame,(x,y),3,(0,0,250),-1)
            if blue != "NA":
                x,y = blue
                cv2.circle(frame,(x,y),3,(255,128,0),-1)
            
        
        if count <=2:
            for score, pt, detect in current_detections:
                for score2, pt2, detect2 in prev_detections:
                    distance= dist(pt, pt2)
                    if distance < 50:
                        tracking_objects[track_id] = [score,pt,detect]
                        used_IDs.append(track_id)
                        track_id = track_id + 1
                        for key, item in coord_dict.items():
                            box,mir,red,blue = item
                            if box.contains(Point(pt)) == True:
                                if box in tracking_objects_boxes.keys():
                                    df_check = pd.concat([df_check, pd.DataFrame({"Video": [file_name], "Frame": [count], 
                                                                                 "Box": [key]})], ignore_index=True)
                                else:
                                    tracking_objects_boxes[box] = pt
                        
                            
        else:
            tracking_objects_copy = tracking_objects.copy()
            current_detections_copy = current_detections.copy()
            
            for object_id, box in tracking_objects_copy.items():
                pt2 = box[1]
                object_exists = False
                for score, pt, detect in current_detections_copy:
                    distance= dist(pt, pt2)
                    if distance < 50:
                        tracking_objects[object_id]= [score,pt,detect]
                        object_exists = True
                        for key, item in coord_dict.items():
                            box,mir,red,blur = item
                            if box.contains(Point(pt)) == True:
                                if box in tracking_objects_boxes.keys():
                                    df_check = pd.concat([df_check, pd.DataFrame({"Video": [file_name], "Frame": [count], 
                                                                                 "Box": [key]})], ignore_index=True)
                                else:
                                    tracking_objects_boxes[box] = pt
                        if [score, pt, detect] in current_detections:
                            current_detections.remove([score, pt, detect])
                        continue
                
                if not object_exists:
                    past1[object_id] = tracking_objects[object_id]
                    tracking_objects.pop(object_id)
            
            for score, pt, detect in current_detections:
                found = False
                for past in past_list:
                    past_copy = past.copy()
                    if found == True:
                        break
                    
                    for object_id, box in past_copy.items():
                        pt2 = box[1]
                        distance = dist(pt, pt2)
                        
                        if distance < 50:
                            tracking_objects.update({object_id: [score,pt,detect]})
                            found = True
                            past.pop(object_id)
                            continue
                        
                        if found == True:
                            break
                
                if found == False:
                    tracking_objects[track_id] = [score,pt,detect]
                    used_IDs.append(track_id)
                    track_id = track_id + 1
                    for key, item in coord_dict.items():
                            box,mir,red,blue = item
                            if box.contains(Point(pt)) == True:
                                if box in tracking_objects_boxes.keys():
                                    df_check = pd.concat([df_check, pd.DataFrame({"Video": [file_name], "Frame": [count], 
                                                                                 "Box": [key]})], ignore_index=True)
                                else:
                                    tracking_objects_boxes[box] = pt
        
        
        past10 = past9
        past9 = past8
        past8 = past7
        past7 = past6
        past6 = past5
        past5 = past4
        past4 = past3
        past3 = past2
        past2 = past1
        past1 = {}
        past_list = [past1, past2, past3, past4, past5, past6, past7, past8, past9, past10]
        
        track_objects_clear = []
        
        box_check = {}
        for object_id, box in tracking_objects.items():
            score = box[0]
            pt = box[1]
            item = box[2]
            (x,y,w,h) = item
            poly_fish = Polygon([(x,y), (x,y+h), (x+w,y+h), (x+w,y)])
            cv2.putText(frame,str((object_id, score)),(int(pt[0]),int(pt[1])-7),0,1,(0,120,120),2)
            
            for key, item in coord_dict.items():
                box,mir,red,blue = item
                box_nr = 0 # this is so that if outside box has nr 0
                if box.contains(Point(pt)) == True:
                    box_nr = key
                    if mir.contains(Point(pt)) == True:
                        by_mirror = "Yes"
                    else: 
                        by_mirror = "No"
                    if img_name in lid_vids:
                        by_lid_pt = "No"
                        by_lid_box = "No"
                        if red != "NA":
                            x,y = red
                            red_pol = Polygon([(float(x-buffer),float(y-buffer)),(float(x+buffer),float(y-buffer)),
                                        (float(x+buffer),float(y+buffer)),(float(x-buffer),float(y+buffer))])
                            if poly_fish.intersects(red_pol) == True:
                                by_lid_box = "Red"
                            if red_pol.contains(Point(pt)) == True:
                                by_lid_pt = "Red"
                        if blue != "NA":
                            x,y = blue
                            blue_pol = Polygon([(float(x-buffer),float(y-buffer)),(float(x+buffer),float(y-buffer)),
                                        (float(x+buffer),float(y+buffer)),(float(x-buffer),float(y+buffer))])
                            if poly_fish.intersects(blue_pol) == True:
                                by_lid_box = "Blue"
                            if blue_pol.contains(Point(pt)) == True:
                                by_lid_pt = "Blue"
                    else:
                        by_lid_pt = "NA"
                        by_lid_box = "NA"
                    
                    temp = [object_id, box_nr, pt[0], pt[1], by_mirror, by_lid_pt, by_lid_box, score]
                    track_objects_clear.append(temp)
                    if box_nr in object_trail.keys():
                        object_trail[box_nr].append(pt)
                        box_check.update({box_nr:"old"})
                    else:
                        object_trail.update({box_nr:[pt]})
                        box_check.update({box_nr:"new"})
                    break
        
        print("Frame", count)
        
        df_coords = pd.DataFrame(track_objects_clear, columns = ["Fish", "Box", "x_coord", "y_coord", "Mirror", "Lid_pt", "Lid_box", "Certainty"])
        df_coords["Frame"] = count
        df_coords["Video"] = file_name
        df_coords["Width"] = width
        df_coords["Frames"] = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        df = pd.concat([df, df_coords], ignore_index=True)
        
        cv2.namedWindow("Fish", cv2.WINDOW_NORMAL)
        cv2.imshow("Fish",frame)
        
        prev_detections= current_detections.copy()
        
        if count == int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
            cv2.imwrite(dest_backup, frame)
            for box_id, pt_list in object_trail.items():
                if len(pt_list) > 1:
                    pts = np.array(pt_list, np.int32)
                    cv2.polylines(frame,[pts],False,(0,255,255))
                    
            cv2.imwrite(dest_img, frame)
            
        
        key= cv2.waitKey(1)
        if key == 27:
            break
    
    cap.release()
    
    if os.path.exists("OF_mirror/OF_mirror_data.csv"):
        df.to_csv("OF_mirror/OF_mirror_data.csv", index=False, header=False, mode="a")
    else:
        df.to_csv("OF_mirror/OF_mirror_data.csv", index=False)
    dest_done = ("videos/analysed/OF_mir/" + os.path.basename(video))
    shutil.move(video, dest_done)
    
    if len(df_check) > 0:
        if os.path.exists("OF_mirror/Dupes_in_boxes.csv"):
            df_check.to_csv("OF_mirror/Dupes_in_boxes.csv", index=False, header=False, mode="a")
        else:
            df_check.to_csv("OF_mirror/Dupes_in_boxes.csv", index=False)
    
    print(time.localtime()[3], ":", time.localtime()[4], ":", time.localtime()[5])
    
    cv2.destroyAllWindows()

print("done")